In [1]:
import numpy as np
import pandas as pd
from sklearn import svm, preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.grid_search import GridSearchCV

IMG_SIZE = 50
FEATURES_PATH = r'D:\LICENTA\processed_data\size_{size1}x{size2}\biz_to_features.npy'.format(size1=IMG_SIZE, size2=IMG_SIZE)

c:\users\eduar\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
c:\users\eduar\appdata\local\programs\python\python36\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
train_data = np.load(FEATURES_PATH)

for i in range(len(train_data)):
    train_data[:,1][i] = train_data[:,1][i].reshape(512,)

In [3]:
business_ids = train_data[:,0]

In [4]:
df = pd.read_csv('train.csv')
df.dropna(axis=0, inplace=True)

train_ids = df['business_id'].values

In [5]:
def convert_labels_to_array():
    labels = []
    
    for str_label in df['labels'].values:
        str_label = str_label.split(' ')
        str_labels = [int(x) for x in str_label if len(x)>0]
                
        labels.append(str_labels)
    return np.array(labels)

In [6]:
new_train_data = train_data[:,1]

print(new_train_data.shape)

X = []

for i in range(len(new_train_data)):
    X.append(new_train_data[i])
    
X = np.array(X)

(1996,)


In [7]:
y = convert_labels_to_array()
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(y)

In [8]:
print('X: ', X.shape)
print('y: ', y.shape)

X:  (1996, 512)
y:  (1996, 9)


In [9]:
random_state = np.random.RandomState(0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)

print('X_train: ',X_train.shape)
print('X_test: ', X_test.shape)
print('y_train: ', y_train.shape)
print('y_test: ', y_test.shape)

X_train:  (1596, 512)
X_test:  (400, 512)
y_train:  (1596, 9)
y_test:  (400, 9)


In [36]:
%%time

clf = OneVsRestClassifier(svm.SVC(kernel='rbf'), n_jobs=-1)
X_train = preprocessing.scale(X_train)
clf.fit(X_train, y_train)

X_test = preprocessing.scale(X_test)
y_pred = clf.predict(X_test)
# parameters = {
#     "estimator__C": [1,2,4,8],
#     "estimator__kernel": ["poly","rbf"],
#     "estimator__degree":[1, 2, 3, 4],
# }

# model_tunning = GridSearchCV(clf, param_grid=parameters,
#                              scoring='f1_micro', n_jobs=-1)
# model_tunning.fit(X, y)


c:\users\eduar\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
c:\users\eduar\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\data.py:181: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


Wall time: 17.2 s


In [38]:
# print(model_tunning.best_score_)
# print(model_tunning.best_params_)

from sklearn.metrics import f1_score

print('F1 Score: ', f1_score(y_test, y_pred, average='micro'))
print('F1 Individual Score: ', f1_score(y_test, y_pred, average=None))

F1 Score:  0.632669322709
F1 Individual Score:  [ 0.          0.47878788  0.65084746  0.60721063  0.          0.76923077
  0.8005997   0.08333333  0.7496063 ]


c:\users\eduar\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
